In [1]:
import pandas as pd

# load cleaned data
df = pd.read_csv('../data/cleaned/cleaned_retail.csv')

# fix datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalAmount,InvoiceMonth
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,2009-12
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,2009-12
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,2009-12
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8,2009-12
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,2009-12


In [2]:
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
snapshot_date


Timestamp('2011-12-10 12:50:00')

In [3]:
rfm = (
    df
    .groupby('Customer ID')
    .agg({
        'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency (days)
        'Invoice': 'nunique',                                     # Frequency (orders)
        'TotalAmount': 'sum'                                      # Monetary (spend)
    })
)

rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()


,Recency,Frequency,Monetary
Customer ID,,,
12346,326,12,77556.46
12347,2,8,5633.32
12348,75,5,2019.40
12349,19,4,4428.69
12350,310,1,334.40


In [4]:
rfm.describe()


,Recency,Frequency,Monetary
count,5878.000000,5878.000000,5878.000000
mean,201.331916,6.289384,3018.616737
std,209.338707,13.009406,14737.731040
min,1.000000,1.000000,2.950000
25%,26.000000,1.000000,348.762500
50%,96.000000,3.000000,898.915000
75%,380.000000,7.000000,2307.090000
max,739.000000,398.000000,608821.650000


In [5]:
# create percentile ranks
rfm['R_rank'] = rfm['Recency'].rank(pct=True)
rfm['F_rank'] = rfm['Frequency'].rank(pct=True)
rfm['M_rank'] = rfm['Monetary'].rank(pct=True)

rfm[['R_rank', 'F_rank', 'M_rank']].head()


,R_rank,F_rank,M_rank
Customer ID,,,
12346,0.710786,0.881507,0.996938
12347,0.023222,0.793722,0.901667
12348,0.456958,0.663066,0.719973
12349,0.196836,0.591102,0.869684
12350,0.698962,0.138142,0.239707


In [6]:
# Recency: lower is better (reverse scale)
rfm['R'] = pd.cut(
    rfm['R_rank'],
    bins=[0, 0.25, 0.5, 0.75, 1],
    labels=[4, 3, 2, 1]
)

# Frequency: higher is better
rfm['F'] = pd.cut(
    rfm['F_rank'],
    bins=[0, 0.25, 0.5, 0.75, 1],
    labels=[1, 2, 3, 4]
)

# Monetary: higher is better
rfm['M'] = pd.cut(
    rfm['M_rank'],
    bins=[0, 0.25, 0.5, 0.75, 1],
    labels=[1, 2, 3, 4]
)

rfm[['R', 'F', 'M']].head()


,R,F,M
Customer ID,,,
12346,2,4,4
12347,4,4,4
12348,3,3,3
12349,4,3,4
12350,2,1,1


In [7]:
rfm['RFM_Score'] = (
    rfm['R'].astype(str) +
    rfm['F'].astype(str) +
    rfm['M'].astype(str)
)

rfm[['RFM_Score']].head()


,RFM_Score
Customer ID,
12346,244
12347,444
12348,333
12349,434
12350,211


In [8]:
rfm[['R', 'F', 'M']].nunique()


R    4
F    4
M    4
dtype: int64

In [9]:
def rfm_segment(row):
    if row['R'] == 4 and row['F'] >= 3 and row['M'] >= 3:
        return 'Champions'
    elif row['F'] >= 3:
        return 'Loyal'
    elif row['R'] <= 2 and row['F'] <= 2:
        return 'At Risk'
    else:
        return 'Others'

rfm['Segment'] = rfm.apply(rfm_segment, axis=1)


In [10]:
rfm['Segment'].value_counts()


Segment
At Risk      2190
Loyal        1647
Others       1041
Champions    1000
Name: count, dtype: int64

In [11]:
segment_pct = (
    rfm['Segment']
    .value_counts(normalize=True) * 100
)

segment_pct


Segment
At Risk      37.257571
Loyal        28.019735
Others       17.710105
Champions    17.012589
Name: proportion, dtype: float64

In [12]:
rfm['Churned'] = rfm['Recency'] > 90


In [13]:
rfm['Churned'].value_counts()


Churned
True     2989
False    2889
Name: count, dtype: int64

In [14]:
churn_rate = rfm['Churned'].mean() * 100
churn_rate


np.float64(50.8506294658047)

In [15]:
rfm.groupby('Segment')['Churned'].mean() * 100


Segment
At Risk      100.000000
Champions      0.000000
Loyal         47.419551
Others         1.729107
Name: Churned, dtype: float64

In [16]:
rfm.head()


,Recency,Frequency,Monetary,R_rank,F_rank,M_rank,R,F,M,RFM_Score,Segment,Churned
Customer ID,,,,,,,,,,,,
12346,326,12,77556.46,0.710786,0.881507,0.996938,2,4,4,244,Loyal,True
12347,2,8,5633.32,0.023222,0.793722,0.901667,4,4,4,444,Champions,False
12348,75,5,2019.40,0.456958,0.663066,0.719973,3,3,3,333,Loyal,False
12349,19,4,4428.69,0.196836,0.591102,0.869684,4,3,4,434,Champions,False
12350,310,1,334.40,0.698962,0.138142,0.239707,2,1,1,211,At Risk,True


In [17]:
rfm.to_csv('../data/cleaned/rfm_churn_final.csv')


In [18]:
pd.read_csv('../data/cleaned/rfm_churn_final.csv').head()


,Customer ID,Recency,Frequency,Monetary,R_rank,F_rank,M_rank,R,F,M,RFM_Score,Segment,Churned
0,12346,326,12,77556.46,0.710786,0.881507,0.996938,2,4,4,244,Loyal,True
1,12347,2,8,5633.32,0.023222,0.793722,0.901667,4,4,4,444,Champions,False
2,12348,75,5,2019.40,0.456958,0.663066,0.719973,3,3,3,333,Loyal,False
3,12349,19,4,4428.69,0.196836,0.591102,0.869684,4,3,4,434,Champions,False
4,12350,310,1,334.40,0.698962,0.138142,0.239707,2,1,1,211,At Risk,True
